## Neural network integrators
Let $q(t)=(x(t),p(t))\in\mathbb{R}^{2d}$ be the state vector. Two methods are implemented to advance this state vector in time:

### Multistep neural network
The `MultistepNNIntegrator` class in [nn_integrator.py](./src/nn_integrator.py) is used to implement a neural network based integrator with a $S$-step method. This is integrating a given $d$-dimensional system $\frac{dq(t)}{dt}=\mathcal{N}(q(t))$. The underlying neural model implements the mapping

$$
q^{(t-(S-1)\Delta t)},\dots,q^{(t-\Delta t)},q^{(t)} \mapsto q^{(t+\Delta t)}
$$

Internally this is realised by mapping the $B\times S \times d$ tensor $X$ to the $B\times d$ tensor $y$, where $B$ is the minibatch-size. This mapping is of the following form:

$$
y_{b,j} = X_{b,S-1,j} + \Delta t \cdot \Phi_{bj}(X)
$$

where $\Phi$ is a dense neural network. Note that for each batch index $b$, $X_{b,S-1,\cdot}$ is simply the vector $q^{(t)}_b$, i.e. we assume that $q^{(t+\Delta t)}$ is $q^{(t)}$ plus $\Delta t$ times some correction. The neural network $\Phi$ can take different form:
* it can simply be a set of dense layers or
* it can be a two-layer LSTM network, followed by a dense layer as in [Kadupitiya et al. (2020)](https://arxiv.org/abs/2004.06493)

### Hamiltonian neural network integrator
Alternatively, the `HamiltonianVerletNNIntegrator` in [nn_integrator.py](./src/nn_integrator.py) implements a single-step Stoermer-Verlet method for a Hamiltonian system, following the ideas in [Greydanus et al. (2019)](https://arxiv.org/abs/1906.01563). In this case the update $q^{(t)}\mapsto q^{(t+\Delta t)}$ takes the form:

$$
\begin{aligned}
p^{(t+\Delta t/2)} &= p^{(t)} - \frac{\Delta t}{2} \frac{\partial V}{\partial x}\left(x^{(t)}\right)\\[1ex]
x^{(t+\Delta t)} &= x^{(t)} + \Delta t \frac{\partial T}{\partial p}\left(p^{(t+\Delta t/2)}\right)\\[1ex]
p^{(t+\Delta t)} &= p^{(t)} - \frac{\Delta t}{2} \frac{\partial V}{\partial x}\left(x^{(t+\Delta t)}\right)
\end{aligned}
$$

Assuming that the Hamiltonian $H(x,p) = T(p) + V(x)$ is separable, the kinetic energy $T(p)$ and potential energy $V(x)$ are represented by neural networks.

## Dynamical systems

![Dynamical systems](figures/DynamicalSystems.svg)

The Hamiltonians of the considered dynamical system are written down explicitly in the following sections. For further details, also see the implementation in [dynamical_system.py](./src/dynamical_system.py).

### Harmonic oscillator

$$
H(q,p) = \frac{p^2}{2m} + \frac{1}{2}k_{\text{spring}}q^2
$$

### Coupled harmonic oscillators

$$
H(q,p) = \frac{p_0^2}{2m_0} + \frac{p_1^2}{2m_1}
    + \frac{1}{2}k_{\text{spring},0} q_0^2 + \frac{1}{2}k_{\text{spring},1}q_1^2
    + k_{\text{spring},c} q_0 q_1
$$

### Double pendulum
The coordinates $q_0$, $q_1$ are the angles of the two rods relative to the vertical.

$$
\begin{aligned}
H(q,p) &= (m_0 + m_1) g L_0 (1-\cos(q_0)) + m_1 g L_1 (1-\cos(q_1))\\
       &+ \left(L_1^2 p_0^2 + \left(1+\frac{m_0}{m_1}\right) L_0^2 p_1^2 - 2 L_0 L_1 p_0 p_1\cos(q_0-q_1) \right)
                \frac{\kappa^2}{2L_0L_1}
\end{aligned}
$$

with $\kappa^{-1} = L_0 L_1 (m_0 + m_1\sin^2(q_0-q_1))$ and the canonical momenta

$$
\begin{aligned}
  p_0 &= (m_0+m_1) L_0^2\dot{q}_0 + m_1 L_0 L_1\dot{q}_1\cos(q_0-q_1)\\
  p_1 &= m_1 L_1^2\dot{q}_1 + m_1 L_0 L_1\dot{q}_0\cos(q_0-q_1)
\end{aligned}
$$

### Coupled pendulums
Again, the coordinates $q_0$, $q_1$ are the angles of the two rods relative to the vertical. Further, write 
$\boldsymbol{x}_j = L(\sin(q_j),-\cos(q_j)) \in\mathbb{R}^2$ for the position if of the $j$-th mass in 2d space.

$$
\begin{aligned}
H(q,p) &= m g L \left( (1-\cos(q_0)) + (1-\cos(q_1)) \right) + \frac{1}{2}k_{\text{spring}} \left(|\boldsymbol{x}_0-\boldsymbol{x}_1|-d\right)^2\\
       &+ \frac{1}{2 m L^2}\left(p_0^2 + p_1^2\right)
\end{aligned}
$$